# Swift SkyMap

What follows is a prototype to sum Swift-XRT images.
*Photon Counting* mode images will be used since they are simpler to deal with.

The pipeline start from a table of Swift observations, at file `SwiftXrt_master.csv`.
This file does not provide all the pointings, but a sub-sample considering the time exposure during `PC` mode.
Although not relevant, the table was extracted from ASDC's Swift database on July 7, 2017.

The pipeline goes like:

* select overlapping observations
* download observation files
* sum event-images
* sum exposure-maps

## Read data

First, get a glance of the file content/format (first rows, since it is an ascii/csv file).

Then, use Pandas to load the table.

In [27]:
!head -n3 ../SwiftXrt_master.csv

NAME;ORIG_TARGET_ID;TARGET_ID;RA;DEC;START_TIME;STOP_TIME;ORIG_OBS_SEGM;OBS_SEGMENT;ORIG_OBSID;OBSID;XRT_EXPOSURE;XRT_EXPO_PC;ARCHIVE_DATE;PROCESSING_DA;PROCESSING_VE;BLA
SAFE3-CURR;60002;60002;0.640524;0.278400;20/11/2004;21/11/2004;0;0;00060002000;00060002000;0;0;27/11/2004;HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006;14/11/2006;3.7.6
SAFE5-CURR;60004;60004;0.640524;0.278400;21/11/2004;21/11/2004;0;0;00060004000;00060004000;0;0;28/11/2004;HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006;12/11/2006;3.7.6


In [20]:
import pandas
pandas.options.display.max_colwidth = 300

In [26]:
table_master = pandas.read_csv('../SwiftXrt_master.csv', sep=';', header=0)
table_master

/home/chbrandt/.conda/envs/booq/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,NAME,ORIG_TARGET_ID,TARGET_ID,RA,DEC,START_TIME,STOP_TIME,ORIG_OBS_SEGM,OBS_SEGMENT,ORIG_OBSID,OBSID,XRT_EXPOSURE,XRT_EXPO_PC,ARCHIVE_DATE,PROCESSING_DA,PROCESSING_VE,BLA
0,SAFE3-CURR,60002,60002,0.640524,0.278400,20/11/2004,21/11/2004,0,0,60002000,60002000,0,0,27/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
1,SAFE5-CURR,60004,60004,0.640524,0.278400,21/11/2004,21/11/2004,0,0,60004000,60004000,0,0,28/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,12/11/2006,3.7.6
2,SAFE1-NEW,60006,60006,0.640524,0.278400,21/11/2004,21/11/2004,0,0,60006000,60006000,0,0,28/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
3,SAFE3-IMAGING,60008,60008,0.640524,0.278400,21/11/2004,21/11/2004,0,0,60008000,60008000,0,0,28/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
4,SAFE5-IMAGING,60010,60010,0.640524,0.278400,21/11/2004,22/11/2004,0,0,60010000,60010000,0,0,28/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
5,NON-SCIENCE DATA,60012,60012,0.640524,0.278400,22/11/2004,22/11/2004,0,0,60012000,60012000,0,0,29/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
6,NON-SCIENCE DATA,60014,60014,0.640524,0.278400,22/11/2004,22/11/2004,0,0,60014000,60014000,0,0,29/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
7,NON-SCIENCE DATA,60016,60016,0.640524,0.278400,22/11/2004,22/11/2004,0,0,60016000,60016000,0,0,29/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
8,NON-SCIENCE DATA,60018,60018,0.640524,0.278400,22/11/2004,23/11/2004,0,0,60018000,60018000,0,0,29/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
9,NON-SCIENCE DATA,60020,60020,0.640524,0.278400,23/11/2004,23/11/2004,0,0,60020000,60020000,0,0,30/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6


## Select table entries

*Criteria* to select the observation from such table should be further explored, sure.
But for this prototype we may simply chose an object, and filter all observations accordingly.

In [24]:
object_select = 'GRB110223A'
#object_select = 'BZBJ1134-1729'
# object_select = '1ES1218+304'
#object_select ='GRB090813'
table_object = table_master.loc[table_master.NAME==object_select]

In [25]:
table_object

,NAME,ORIG_TARGET_ID,TARGET_ID,RA,DEC,START_TIME,STOP_TIME,ORIG_OBS_SEGM,OBS_SEGMENT,ORIG_OBSID,OBSID,XRT_EXPOSURE,XRT_EXPO_PC,ARCHIVE_DATE
52310,GRB110223A,446674,446674,345.895727,87.558482,23/2/2011,23/2/2011,0,0,446674000,446674000,583,541,2/3/2011
52312,GRB110223A,446674,446674,345.984650,87.581965,23/2/2011,23/2/2011,1,1,446674001,446674001,1844,1841,2/3/2011
52328,GRB110223A,446674,446674,346.218620,87.538314,24/2/2011,24/2/2011,2,2,446674002,446674002,9769,9743,3/3/2011
52345,GRB110223A,446674,446674,346.107831,87.573624,24/2/2011,24/2/2011,3,3,446674003,446674003,2324,2321,3/3/2011
52359,GRB110223A,446674,446674,346.010043,87.577552,25/2/2011,25/2/2011,4,4,446674004,446674004,2987,2982,4/3/2011
52367,GRB110223A,446674,446674,346.315446,87.573754,25/2/2011,26/2/2011,5,5,446674005,446674005,7626,7626,4/3/2011
52415,GRB110223A,446674,446674,345.861283,87.570744,26/2/2011,26/2/2011,6,6,446674006,446674006,9551,9551,5/3/2011
52434,GRB110223A,446674,446674,345.860193,87.583352,26/2/2011,27/2/2011,7,7,446674007,446674007,6253,6253,5/3/2011
52467,GRB110223A,446674,446674,345.951735,87.584481,27/2/2011,28/2/2011,8,8,446674008,446674008,9811,9811,6/3/2011
52539,GRB110223A,446674,446674,346.019558,87.583121,1/3/2011,1/3/2011,9,9,446674009,446674009,5789,5789,8/3/2011


### Check selection

In [55]:
table_object.describe()

,ORIG_TARGET_I,TARGET_ID,RA,DEC,ORIG_OBS_SEGM,OBS_SEGMENT,ORIG_OBSID,OBSID,XRT_EXPOSURE,XRT_EXPO_PC
count,10.0,10.0,10.000000,10.000000,10.00000,10.00000,1.000000e+01,1.000000e+01,10.0000,10.000000
mean,359884.0,359884.0,226.279201,88.588000,6.30000,6.30000,3.598840e+08,3.598840e+08,9066.1201,8980.995800
std,0.0,0.0,0.908315,0.018235,3.40098,3.40098,3.400980e+00,3.400980e+00,6579.6416,6321.725057
min,359884.0,359884.0,224.551432,88.565102,0.00000,0.00000,3.598840e+08,3.598840e+08,3858.0420,3857.993000
25%,359884.0,359884.0,225.925263,88.575110,4.25000,4.25000,3.598840e+08,3.598840e+08,5776.4605,5776.458500
50%,359884.0,359884.0,226.732657,88.585358,6.50000,6.50000,3.598840e+08,3.598840e+08,7638.3200,7638.225500
75%,359884.0,359884.0,226.826558,88.594839,8.75000,8.75000,3.598840e+08,3.598840e+08,8601.9745,8601.767000
max,359884.0,359884.0,227.063646,88.627077,11.00000,11.00000,3.598840e+08,3.598840e+08,27058.1830,26207.813000


There are 5 selected entries, should be enough for this test.
We now have to download the respective files.

The Swift archive is organized by observation's start date (column `START_TIME`) and id (`OBSID`):
`ftp://legacy.gsfc.nasa.gov/swift/data/obs/`**START_TIME**`/`**OBSID**`/`.

In [39]:
def swift_archive_obs_path(date,obsid):
    '''
    Format `date` and `obsid` information for swift archive
    '''
    def extract_date(archive_date):
        '''
        Extract year/month from swift date format

        Example:
        "13/06/26 23:59:12" --> "2013_06"
        '''
        from datetime import datetime
        try:
            dt = datetime.strptime(archive_date,'%y/%m/%d %H:%M:%S')
        except:
            print(archive_date)
            return None
        year_month = '{:4d}_{:02d}'.format(dt.year,dt.month)
        return year_month

    dtf = extract_date(date)
    if dtf is None:
        return None
    obs = '{:011d}'.format(obsid)
    return '{}/{}'.format(dtf,obs)



fileout='1ES_object_observations.csv'
table_object.apply(lambda x:swift_archive_obs_path(x.START_TIME,x.OBSID), axis=1).to_csv(fileout, index=False)
!cat $fileout

2017_02/00030376079
2016_03/00030376060
2017_02/00030376080
2016_02/00030376059
2017_04/00030376087
2016_05/00030376072
2016_03/00030376062
2017_04/00030376086
2016_03/00030376063
2016_05/00030376073
2017_01/00030376076
2016_04/00030376067
2016_04/00030376071
2014_03/00030376033
2014_02/00030376032
2017_03/00030376083
2016_04/00030376069
2016_03/00030376061
2017_05/00030376089
2016_04/00030376068
2006_05/00030376006
2006_05/00030376005
2016_04/00030376070
2017_03/00030376084
2016_02/00030376057
2016_02/00030376058
2017_04/00030376085
2017_04/00030376088
2017_01/00030376075
2016_02/00030376056
2017_01/00030376078
2017_01/00030376077
2016_12/00030376074
2006_05/00030376004
2006_05/00030376003
2006_03/00030376001
2006_03/00030376002


## Download observation files

To download the files, let's use a script in the background (the script is written in Bash and calls `wget`).

$ `./download_obslist.sh <fileout>`

After a couple of minutes, we should have the files like shown below.
(For some reason, the "date" part of observation `00085333007` is associated with its `ARCHIVE_DATE`, i.e, `2015_05` and was not downloaded now, which should not be a problem for us, anyway.)

In [41]:
!ls swift/data/obs/*

swift/data/obs/2011_02:
00446674000  00446674003  00446674006  00446674008
00446674002  00446674004  00446674007

swift/data/obs/2011_03:
00446674011  00446674012  00446674019

swift/data/obs/2012_08:
00091401002  00091401003

swift/data/obs/2012_11:
00091401005  00091401006  00091401007  00091401008  00091401009

swift/data/obs/2013_01:
00091401011  00091401012

swift/data/obs/2013_03:
00091401015

swift/data/obs/2014_11:
00085333001

swift/data/obs/2014_12:
00085333002

swift/data/obs/2015_04:
00085333004

swift/data/obs/2015_07:
00085333009


## Create the image stacks

There is also a shell script for doing this task(s) in the background.

The shell command line goes like:
```
source setup_ximage_files.fsh 

EVENTSFILE='object_events.txt'
EXMAPSFILE='object_exmaps.txt'

# Create two files with filenames list of event-images and exposure-maps
#
create_obsfilelist $PWD 'object_observations.csv' $EVENTSFILE $EXMAPSFILE

# Sum the event-images
#
script_xselect_sum 'object' $EVENTSFILE 'events_sum.xcm' 'evts'

# Sum the exposure-maps
#
script_ximage_sum 'object' $EXMAPSFILE 'exmaps_sum.xco' 'expo'
```

### Run XImage scripts

Now it is time to run `events_sum.xco` and `exmaps_sum.xco` on `ximage` to have the final/stacked images.

For clarity, the content of the above `xco` files are *echo'd* below...

In [45]:
!cat 1ES/events_sum.xcm

xsel
read ev sw00030376079xpcw3po_cl.evt.gz
tmp_dir/
yes
read ev sw00030376060xpcw3po_cl.evt.gz
extract ev
read ev sw00030376080xpcw3po_cl.evt.gz
extract ev
read ev sw00030376059xpcw3po_cl.evt.gz
extract ev
read ev sw00030376087xpcw3po_cl.evt.gz
extract ev
read ev sw00030376072xpcw3po_cl.evt.gz
extract ev
read ev sw00030376062xpcw3po_cl.evt.gz
extract ev
read ev sw00030376086xpcw3po_cl.evt.gz
extract ev
read ev sw00030376063xpcw3po_cl.evt.gz
extract ev
read ev sw00030376073xpcw3po_cl.evt.gz
extract ev
read ev sw00030376076xpcw3po_cl.evt.gz
extract ev
read ev sw00030376067xpcw3po_cl.evt.gz
extract ev
read ev sw00030376071xpcw3po_cl.evt.gz
extract ev
read ev sw00030376033xpcw3po_cl.evt.gz
extract ev
read ev sw00030376032xpcw3po_cl.evt.gz
extract ev
read ev sw00030376083xpcw3po_cl.evt.gz
extract ev
read ev sw00030376069xpcw3po_cl.evt.gz
extract ev
read ev sw00030376061xpcw3po_cl.evt.gz
extract ev
read ev sw00030376089xpcw3po_cl.evt.gz
extract ev
read ev sw00030376068xpcw3po_cl.evt.gz
extr

In [46]:
!cat 1ES/exmaps_sum.xco

cpd  object_sum_expo.gif/gif
read/size=800  /input/example/1ES/swift/data/obs/2017_02/00030376079/xrt/products/sw00030376079xpc_ex.img.gz
read/size=800  /input/example/1ES/swift/data/obs/2016_03/00030376060/xrt/products/sw00030376060xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2017_02/00030376080/xrt/products/sw00030376080xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2016_02/00030376059/xrt/products/sw00030376059xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2017_04/00030376087/xrt/products/sw00030376087xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2016_05/00030376072/xrt/products/sw00030376072xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2016_03/00030376062/xrt/products/sw00030376062xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2017_04/00030376086/xrt/products/

Running the above scripts, will give us the iamges below.
```
xselect < events_sum.xcm
ximage < exmaps_sum.xco
```

### Event-images stack:

![Events](object_sum_evts.gif)

### Exposure-maps stack:

![Exposure maps](object_sum_expo.gif)

## Who's inside Stripe82?

In [28]:
from numpy import logical_or,logical_and
ra_idx = logical_or(table_master['RA'].between(0,60),table_master['RA'].between(330,360))
dec_idx = table_master['DEC'].between(-1.5,1.5)
table_stripe82 = table_master.loc[logical_and(ra_idx,dec_idx)]
table_stripe82

,NAME,ORIG_TARGET_ID,TARGET_ID,RA,DEC,START_TIME,STOP_TIME,ORIG_OBS_SEGM,OBS_SEGMENT,ORIG_OBSID,OBSID,XRT_EXPOSURE,XRT_EXPO_PC,ARCHIVE_DATE,PROCESSING_DA,PROCESSING_VE,BLA
0,SAFE3-CURR,60002,60002,0.640524,0.278400,20/11/2004,21/11/2004,0,0,60002000,60002000,0,0,27/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
1,SAFE5-CURR,60004,60004,0.640524,0.278400,21/11/2004,21/11/2004,0,0,60004000,60004000,0,0,28/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,12/11/2006,3.7.6
2,SAFE1-NEW,60006,60006,0.640524,0.278400,21/11/2004,21/11/2004,0,0,60006000,60006000,0,0,28/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
3,SAFE3-IMAGING,60008,60008,0.640524,0.278400,21/11/2004,21/11/2004,0,0,60008000,60008000,0,0,28/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
4,SAFE5-IMAGING,60010,60010,0.640524,0.278400,21/11/2004,22/11/2004,0,0,60010000,60010000,0,0,28/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
5,NON-SCIENCE DATA,60012,60012,0.640524,0.278400,22/11/2004,22/11/2004,0,0,60012000,60012000,0,0,29/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
6,NON-SCIENCE DATA,60014,60014,0.640524,0.278400,22/11/2004,22/11/2004,0,0,60014000,60014000,0,0,29/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
7,NON-SCIENCE DATA,60016,60016,0.640524,0.278400,22/11/2004,22/11/2004,0,0,60016000,60016000,0,0,29/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
8,NON-SCIENCE DATA,60018,60018,0.640524,0.278400,22/11/2004,23/11/2004,0,0,60018000,60018000,0,0,29/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6
9,NON-SCIENCE DATA,60020,60020,0.640524,0.278400,23/11/2004,23/11/2004,0,0,60020000,60020000,0,0,30/11/2004,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,14/11/2006,3.7.6


In [29]:
objects_s82 = table_stripe82.NAME.unique()
objects_s82.sort()
objects_s82

array(['1FGLJ2223.3+0103', '1RXSJ003238.2-010056', '1RXSJ004135.7-012143',
       '1RXSJ004547.2-005034', '1RXSJ004941.8-011657',
       '1RXSJ005350.4-013023', '1RXSJ010227.1-003911',
       '1RXSJ011202.7-004355', '1RXSJ013051.8+012712',
       '1RXSJ013353.5+011343', '1RXSJ014942.9+001504',
       '1RXSJ015105.7-003427', '1RXSJ020146.5+011717',
       '1RXSJ020220.5-010609', '1RXSJ020221.0-010715',
       '1RXSJ020622.7-011754', '1RXS_J000709.8+005328',
       '1RXS_J001821.7+011859', '1RXS_J002147.4+000036',
       '1RXS_J003846.3+003430', '1RXS_J005441.9+000056',
       '1RXS_J010243.0+010805', '1RXS_J010939.6+005943',
       '1RXS_J011704.2+000025', '1RXS_J012202.5+002023',
       '1RXS_J013137.7+003251', '1RXS_J013353.5+011343',
       '1RXS_J014239.9+000514', '1RXS_J014942.9+001504',
       '1RXS_J015910.0+010518', '1RXS_J020002.0+012123',
       '1RXS_J020106.3+003400', '1RXS_J020151.6+012902',
       '1RXS_J020432.4+012208', '1RXS_J021359.6+004245',
       '1RXS_J024241.2+005

### Resolv objects...

In [30]:
def resolve_name(name):
    '''
    Return ICRS position for given object 'name'
    Input:
     - name : str
        Object designation/name
    Output:
     - position : (float,float)
        Tuple with (RA,Dec) coordinates in degree
    '''
    from astropy.coordinates import get_icrs_coordinates as get_coords
    try:
        icrs = get_coords(name)
        pos = (icrs.ra.value,icrs.dec.value)
    except:
        pos = None
    return pos


valid_objects = {}

import re
for obj in objects_s82:
    mat = re.search('J\d',obj)
    if not (mat and mat.start()>0):
        continue
    start = mat.start()
    oprj = re.sub('_','',obj[:start])
    opos = obj[start:]
    obj = '{} {}'.format(oprj,opos)
    print("Resolving object '{}' position".format(obj))
    pos = resolve_name(obj)
    if pos != None:
        print('...position: {!s}'.format(pos))
        valid_objects[obj] = dict(ra=pos[0],dec=pos[1])
    else:
        print('...position NOT resolved')


Resolving object '1FGL J2223.3+0103' position
...position: (335.829, 1.064)
Resolving object '1RXS J003238.2-010056' position
...position: (8.1591667, -1.0155556)
Resolving object '1RXS J004135.7-012143' position
...position: (10.39875, -1.3619444)
Resolving object '1RXS J004547.2-005034' position
...position: (11.4647337, -0.8601794)
Resolving object '1RXS J004941.8-011657' position
...position: (12.4241667, -1.2825)
Resolving object '1RXS J005350.4-013023' position
...position: (13.4580833, -1.5083889)
Resolving object '1RXS J010227.1-003911' position
...position: (15.6096601, -0.6512789)
Resolving object '1RXS J011202.7-004355' position
...position: (18.0168926, -0.7310771)
Resolving object '1RXS J013051.8+012712' position
...position: (22.71875, 1.4525)
Resolving object '1RXS J013353.5+011343' position
...position: (23.4694129, 1.2292108)
Resolving object '1RXS J014942.9+001504' position
...position: (27.4271351, 0.250477)
Resolving object '1RXS J015105.7-003427' position
...positi

...position: (46.1550528, 0.7815829)
Resolving object 'SDSS J030911.64+002358.8' position
...position: (47.2985313, 0.3996808)
Resolving object 'SDSS J031757.07-001008.7' position
...position: (49.487794, -0.1690731)
Resolving object 'SDSS J032213+005513' position
...position NOT resolved
Resolving object 'SDSS J032213.89+005513.4' position
...position: (50.5579027, 0.9203944)
Resolving object 'SDSS J033059.05+0109' position
...position: (52.746183, 1.150054)
Resolving object 'SDSS J034135.63-004032.2' position
...position: (55.3984736, -0.6756301)
Resolving object 'SDSS J110243.20+385152.5' position
...position: (165.6800077, 38.8646077)
Resolving object 'SDSS J222024.58+010931.2' position
...position: (335.1024498, 1.158691)
Resolving object 'SDSS J222032.50+002537.5' position
...position: (335.1354236, 0.4271001)
Resolving object 'SDSS J222944.18-00342' position
...position NOT resolved
Resolving object 'SDSS J2234+0041' position
...position NOT resolved
Resolving object 'SDSS J2238

In [31]:
from pandas import DataFrame
s82_objects = DataFrame(valid_objects).transpose()
s82_objects.index.name = 'object'
s82_objects

,dec,ra
object,,
1FGL J2223.3+0103,1.064000,335.829000
1RXS J000709.8+005328,0.891420,1.791724
1RXS J001821.7+011859,1.316959,4.591857
1RXS J002147.4+000036,0.010000,5.447500
1RXS J003238.2-010056,-1.015556,8.159167
1RXS J003846.3+003430,0.582651,9.699866
1RXS J004135.7-012143,-1.361944,10.398750
1RXS J004547.2-005034,-0.860179,11.464734
1RXS J004941.8-011657,-1.282500,12.424167


In [32]:
s82_objects.to_csv('SwiftXrt_S82.csv', sep=';')